In [12]:
import dynamiqs as dq
import jax
import jax.numpy as jnp # the JAX version of numpy
from matplotlib import pyplot as plt

$$
\begin{aligned}
\hat{H} &= \hat{H}_{\mathrm{2ph}} + \hat{H}_d,\qquad \mathrm{with}\\
\hat{H}_{\mathrm{2ph}} &= g_2 {{}\hat{a}^\dagger}^2 \hat{b} + g_2^* \hat{a}^2 \hat{b}^\dagger,\\
\hat{H}_d &= \epsilon_d^* \hat{b} + \epsilon_d \hat{b}^\dagger.
\end{aligned}
$$

In [9]:
import dynamiqs as dq
dq.set_device('cpu')
na = 20  # number of Fock states
nb = 5
a = dq.destroy(na)
adag = dq.dag(a)
b = dq.destroy(nb)
bdag = dq.dag(b)
g2 = 1
eps = -4
kap = 10
H2ph = g2*adag@adag@b+dq.dag(g2)@a@adag@bdag
Hd = dq.dag(eps)@b+eps@bdag
# With this, we run into a dimensionality problem

ValueError: Qarrays have incompatible Hilbert space dimensions. Got (20,) and (5,).

In [15]:
import dynamiqs as dq
dq.set_device('cpu')

na = 20  # Fock states for subsystem A
nb = 5   # Fock states for subsystem B

# Define identity operators for each subsystem
Ia = dq.eye(na)
Ib = dq.eye(nb)

# Define annihilation operators for each subsystem
a = dq.destroy(na)
b = dq.destroy(nb)

# Embed operators into the composite Hilbert space (na ⊗ nb)
a_tensored = dq.tensor(a, Ib)      # a ⊗ I_b
adag_tensored = dq.dag(a_tensored)
b_tensored = dq.tensor(Ia, b)      # I_a ⊗ b
bdag_tensored = dq.dag(b_tensored)

g2 = 1.0
eps = -4.0

H2ph = g2 * (adag_tensored @ adag_tensored @ b_tensored) + g2.conjugate() * (a_tensored @ a_tensored @ bdag_tensored)

# Drive term: eps (I_a ⊗ b†) + eps* (I_a ⊗ b)
Hd = eps * bdag_tensored + eps.conjugate() * b_tensored

# Full Hamiltonian
H = H2ph + Hd  # Now all terms act on the composite space (na ⊗ nb)

In [21]:
T = 4
psi0 = dq.fock(na,0)
t_save = jnp.linspace(0, T, 50)  # save times

In [24]:
result = dq.sesolve(H, psi0, t_save)
print(result)

ValueError: Terms are not compatible with solver!

https://www.dynamiqs.org/stable/documentation/getting_started/sharp-bits.html